In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp_api
import kerastuner
import numpy as np
import pandas as pd
import os
import json
import datetime
import dill

In [3]:
from tensorflow.keras.layers import (
    TimeDistributed, 
    Dense, 
    Conv1D, 
    MaxPooling1D, 
    Bidirectional, 
    LSTM, 
    Dropout
)

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [5]:
log_dir = "ex"

In [6]:
dataset_name = "SEG_AR"

In [7]:
static_params = {
    'PAST_HISTORY': 16,
    'FUTURE_TARGET': 8,
    'BATCH_SIZE': 512,
    'BUFFER_SIZE': 200000,
    'EPOCHS': 500,
    'VOCAB_SIZE': 16293
 }

In [8]:
hparams = {
    "HP_LSTM_1_UNITS" : 128,
    "HP_LSTM_1_DROPOUT" : 0.0,
    "HP_LEARNING_RATE" : 1e-3,
}

In [9]:
def generate_timeseries(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, n_feature)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        #data.append(dataset[indices])
        labels.append(np.reshape(dataset[i:i+target_size], (target_size, 1)))
        #labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [10]:
train_set = np.genfromtxt("data/SEG_train_set.csv", delimiter="\n", dtype=np.int32)
x_train, y_train = generate_timeseries(train_set, 0, None, static_params["PAST_HISTORY"], static_params["FUTURE_TARGET"])
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().batch(static_params["BATCH_SIZE"]).shuffle(static_params["BUFFER_SIZE"])

In [11]:
val_set = np.genfromtxt("data/SEG_val_set.csv", delimiter="\n", dtype=np.int32)
x_val, y_val = generate_timeseries(val_set, 0, None, static_params["PAST_HISTORY"], static_params["FUTURE_TARGET"])
val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.cache().batch(static_params["BATCH_SIZE"])

In [15]:
class SEGAutoRegressive(keras.Model):
    def __init__(self, units, dropout, output_steps, output_size):
        super().__init__()
        self.output_steps = output_steps
        self.units = units
        self.lstm_cell = keras.layers.LSTMCell(units, dropout=dropout)

        self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
        self.dense = Dense(output_size, activation="softmax")

    @tf.function#(input_signature=[tf.TensorSpec(shape=[None, None, 1], dtype=tf.int32)])
    def warmup(self, inputs):
        onehot_inputs = tf.squeeze(tf.one_hot(inputs, static_params["VOCAB_SIZE"]), axis=2)

        # inputs.shape => (batch, time, features)
        # x.shape => (batch, lstm_units)
        x, *state = self.lstm_rnn(onehot_inputs)

        # predictions.shape => (batch, features)
        prediction = self.dense(x)

        return prediction, state

    @tf.function#(input_signature=[tf.TensorSpec(shape=[None, None, 1], dtype=tf.int32)])
    def call(self, inputs, training=None):
        # Use a TensorArray to capture dynamically unrolled outputs.
        #predictions = []
        predictions = tf.TensorArray(tf.float32, size=self.output_steps, clear_after_read=False)
        # Initialize the lstm state
        prediction, state = self.warmup(inputs)

        # Insert the first prediction
        #predictions.append(prediction)
        predictions = predictions.write(0, prediction)

        # Run the rest of the prediction steps
        for i in tf.range(1, self.output_steps):
            # Use the last prediction as input.
            x = prediction

            # Execute one lstm step.
            x, state = self.lstm_cell(x, states=state, training=training)
            
            # Convert the lstm output to a prediction.
            prediction = self.dense(x)

            # Add the prediction to the output
            #predictions.append(prediction)
            predictions = predictions.write(i, prediction)

        # predictions.shape => (time, batch, features)
        #predictions = tf.stack(predictions)
        predictions = predictions.stack()

        # predictions.shape => (batch, time, features)
        predictions = tf.transpose(predictions, [1, 0, 2])

        return predictions

In [16]:
model = SEGAutoRegressive(
    units=hparams["HP_LSTM_1_UNITS"], dropout=hparams["HP_LSTM_1_DROPOUT"], 
    output_steps=static_params["FUTURE_TARGET"], output_size=static_params["VOCAB_SIZE"])
model.compile(
    optimizer=keras.optimizers.Nadam(hparams["HP_LEARNING_RATE"]),
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy']
)

In [17]:
model.predict(x_train[:2].reshape(2, -1, 1))

Tensor("lstm_cell_1/mul_2:0", shape=(None, 128), dtype=float32)


array([[[6.1445746e-05, 6.1353123e-05, 6.1505591e-05, ...,
         6.1475395e-05, 6.1428320e-05, 6.1368424e-05],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        ...,
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00]],

       [[6.1445746e-05, 6.1353123e-05, 6.1505591e-05, ...,
         6.1475395e-05, 6.1428320e-05, 6.1368424e-05],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.000

In [281]:
model.predict(x_train[:2].reshape(2, -1, 1))

array([[[6.1407009e-05, 6.1159910e-05, 6.1471401e-05, ...,
         6.1185157e-05, 6.1460407e-05, 6.1534498e-05],
        [6.1415267e-05, 6.1199942e-05, 6.1451465e-05, ...,
         6.1217317e-05, 6.1451734e-05, 6.1515988e-05],
        [6.1419305e-05, 6.1232531e-05, 6.1435072e-05, ...,
         6.1244253e-05, 6.1442028e-05, 6.1497434e-05],
        ...,
        [6.1416678e-05, 6.1299492e-05, 6.1402381e-05, ...,
         6.1302126e-05, 6.1414576e-05, 6.1448438e-05],
        [6.1413448e-05, 6.1314473e-05, 6.1395571e-05, ...,
         6.1315688e-05, 6.1407140e-05, 6.1435487e-05],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00]],

       [[6.1407009e-05, 6.1159910e-05, 6.1471401e-05, ...,
         6.1185157e-05, 6.1460407e-05, 6.1534498e-05],
        [6.1415267e-05, 6.1199942e-05, 6.1451465e-05, ...,
         6.1217317e-05, 6.1451734e-05, 6.1515988e-05],
        [6.1419305e-05, 6.1232531e-05, 6.1435072e-05, ...,
         6.124

In [292]:
for x, y in train_data.take(1):
    x_sample = x
model.predict(x)

array([[[6.1394290e-05, 6.1312676e-05, 6.1343890e-05, ...,
         6.1373983e-05, 6.1312516e-05, 6.1396429e-05],
        [6.1401341e-05, 6.1324237e-05, 6.1342740e-05, ...,
         6.1372775e-05, 6.1332474e-05, 6.1393475e-05],
        [6.1404193e-05, 6.1334067e-05, 6.1344050e-05, ...,
         6.1372601e-05, 6.1346393e-05, 6.1390361e-05],
        ...,
        [6.1400817e-05, 6.1354905e-05, 6.1353589e-05, ...,
         6.1373510e-05, 6.1367158e-05, 6.1382583e-05],
        [6.1398219e-05, 6.1359540e-05, 6.1357016e-05, ...,
         6.1373816e-05, 6.1370185e-05, 6.1380808e-05],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00]],

       [[6.1394981e-05, 6.1348357e-05, 6.1331943e-05, ...,
         6.1387371e-05, 6.1324776e-05, 6.1382940e-05],
        [6.1401130e-05, 6.1351318e-05, 6.1334358e-05, ...,
         6.1381717e-05, 6.1344195e-05, 6.1382299e-05],
        [6.1403560e-05, 6.1354571e-05, 6.1338447e-05, ...,
         6.137

In [295]:
history = model.fit(train_data, epochs=1)

....] - ETA: 1:40 - loss: 7.9382 - accuracy: 0.23 79/315 [======>.......................] - ETA: 1:39 - loss: 7.9271 - accuracy: 0.23 80/315 [======>.......................] - ETA: 1:40 - loss: 7.8926 - accuracy: 0.24 81/315 [======>.......................] - ETA: 1:40 - loss: 7.9021 - accuracy: 0.23 82/315 [======>.......................] - ETA: 1:40 - loss: 7.9008 - accuracy: 0.23 83/315 [======>.......................] - ETA: 1:40 - loss: 7.8542 - accuracy: 0.23 84/315 [=======>......................] - ETA: 1:40 - loss: 7.8818 - accuracy: 0.23 85/315 [=======>......................] - ETA: 1:40 - loss: 7.9179 - accuracy: 0.23 86/315 [=======>......................] - ETA: 1:40 - loss: 7.8985 - accuracy: 0.23 87/315 [=======>......................] - ETA: 1:40 - loss: 7.8998 - accuracy: 0.23 88/315 [=======>......................] - ETA: 1:39 - loss: 7.8928 - accuracy: 0.23 89/315 [=======>......................] - ETA: 1:39 - loss: 7.8784 - accuracy: 0.23 90/315 [=======>..........

In [156]:
model.summary()

Model: "seg_auto_regressive_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #
lstm_cell_34 (LSTMCell)      multiple                  8408064
_________________________________________________________________
rnn_34 (RNN)                 multiple                  8408064
_________________________________________________________________
dense_34 (Dense)             multiple                  2101797
Total params: 10,509,861
Trainable params: 10,509,861
Non-trainable params: 0
_________________________________________________________________


In [285]:
tf.saved_model.save(model, "ex/model_4", 
    signatures=model.call.get_concrete_function(tf.TensorSpec(shape=[None, None, 1], dtype=tf.int32, name="input")))

INFO:tensorflow:Assets written to: ex/model_4\assets


In [286]:
new_model = tf.saved_model.load("ex/model_4")

In [287]:
new_model.signatures.items()

ItemsView(_SignatureMap({'serving_default': <tensorflow.python.saved_model.load._WrapperFunction object at 0x00000281F5C235C8>}))

In [294]:
inference = new_model.signatures["serving_default"]
inference(tf.constant(x_train[0].reshape(1, -1, 1)))

{'output_0': <tf.Tensor: shape=(1, 8, 16293), dtype=float32, numpy=
 array([[[6.1407009e-05, 6.1159910e-05, 6.1471401e-05, ...,
          6.1185157e-05, 6.1460407e-05, 6.1534498e-05],
         [6.1415267e-05, 6.1199942e-05, 6.1451465e-05, ...,
          6.1217317e-05, 6.1451734e-05, 6.1515988e-05],
         [6.1419305e-05, 6.1232531e-05, 6.1435072e-05, ...,
          6.1244253e-05, 6.1442028e-05, 6.1497434e-05],
         ...,
         [6.1416678e-05, 6.1299492e-05, 6.1402381e-05, ...,
          6.1302126e-05, 6.1414576e-05, 6.1448438e-05],
         [6.1413448e-05, 6.1314473e-05, 6.1395571e-05, ...,
          6.1315688e-05, 6.1407140e-05, 6.1435487e-05],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00]]], dtype=float32)>}

In [20]:
tf.saved_model.save(model, version_dir)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: version/20200916-205849\assets


In [21]:
model_1 = keras.models.load_model(version_dir) 


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent_v2.LSTMCell object at 0x0000018AB61B91C8> and <tensorflow.python.keras.layers.recurrent_v2.LSTMCell object at 0x0000018AB61BDFC8>).


AssertionError: Some Python objects were not bound to checkpointed values, likely due to changes in the Python program: [<tf.Variable 'lstm_cell_1/recurrent_kernel:0' shape=(128, 512) dtype=float32, numpy=
array([[-0.00730717, -0.0198552 ,  0.06291562, ..., -0.00453906,
        -0.02963837,  0.05556399],
       [-0.08210181,  0.01879025,  0.02236442, ...,  0.07328226,
         0.03787373,  0.01388676],
       [-0.00642601, -0.05339912, -0.00547218, ...,  0.05781174,
        -0.05629285,  0.02388919],
       ...,
       [-0.03962746, -0.01832095,  0.04701049, ...,  0.07243678,
         0.01312423,  0.01554349],
       [-0.01769733, -0.03605518, -0.0195809 , ..., -0.0357281 ,
        -0.01636354,  0.05007046],
       [ 0.03688146,  0.06794242, -0.06881212, ...,  0.04910905,
        -0.02574294,  0.01410268]], dtype=float32)>, <tf.Variable 'lstm_cell_1/bias:0' shape=(512,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0

In [25]:
model.save_weights(version_dir + "/weights/")

In [23]:
model_1.load_weights(version_dir + "weights")

NameError: name 'model_1' is not defined